# MIRI LRS Slit Spectroscopy: Spectral Extraction

### Author, Date

Sarah Kendrew, ESA and STScI MIRI Branch, sarah.kendrew@esa.int</br>
March 2023


### <span style="color:red">Tl;dr?</span>

<span style="color:red">**Want pretty spectra, don't want to read all the words? Grab the abbreviated version of this notebook in script form, miri_lrs_extract_L3.py**. 
</span>


### Spectral extraction issues

The automated spectra extraction in the JWST calibration pipeline is very sensitive to coordinate inaccuracies, and as a result, the extract (``x1d``) products in MAST are often of poor quality. Specifically, the ``extract_1d`` step does not reliably place the extraction aperture in the correct place on the array. The algorithm uses the target coordinates in the metadata and the WCS information to place the aperture at the location of the target; the algorithm itself is actually very effective, but as it relies on accurate coordinate information both in the header and in the data WCS information, it is very sensitive to coordinate registration issues. Reasons for such coordinate issues are e.g.:

* inaccuracies in the guide star coordinates that cause the RA, Dec registration of the field to be ofset
* coordinate tarnsformation inaccuracies when resampling
* target coordinate errors

We are actively working on resolving these issues, and in the mean time are updating the specral extraction process to extract based on detector position rather than WCS location. 

**In this notebook we will demo how to perform a more accurate extraction manually using the JWST pipeline, for LEVEL 3 data**. 


### Aims

This notebook will demonstrate how to get a well-calibrated and extracted spectrum from a Low-Resolution Spectroscopy (LRS) observation of a point source in the slit. We have chosen as example data an observation where the extracted products in MAST are affected by the above issues, and don't look right. 

We are actively working on improving the robustness of the spectral extraction algorithms. In the mean time, this notebook provides a template for how to work around the extraction issues and produce clean, well-extracted spectra. We will provide additional explanation along the way. 


### Assumptions

We will start from the Level 2a data. This is the file ending in ``rate.fits``. This data product has undergone basic detector calibration. The units are DN/s. 

The calibration steps assume the target is an unresolved point source, and that the observation used the ALONG SLIT NOD dither pattern. 

**The custom extraction is applied at the Level 3 stage of the pipeline. We include the Spec2Pipeline to provide some extra info & context, but you can also jump in just for the Spec3Pipeline.**


### Test data

The data used in this notebook is an observation of the Type Ia supernova SN2021aefx, observed by Jha et al in PID 2072 (Obs 1), **where the automated spectral extraction failed**. These data were taken with zero exclusive access period, and published in [Kwok et al 2023](https://ui.adsabs.harvard.edu/abs/2023ApJ...944L...3K/abstract). You can retrieve the data from [this Box folder](https://stsci.box.com/s/i2xi18jziu1iawpkom0z2r94kvf9n9kb), and we recommend you place the files in the ``data/`` folder of this repository, or change the directory settings in the notebook prior to running. 

You can of course just use your own data instead of the demo data. 

### CRDS pmap 1088 update (May 2023)

**The LRS spectral extraction reference file updated in May 2023, starting from CRDS context (pmap) 1088, to address the issue described in this notebook.**

We will set the context explicitly to pmap 1084, so the pipeline outputs demonstrate the problem prior to the fix. You can see the effect of the update by changing the CRDS context number to a more recent version in the cell below. 

If your CRDS context is 1088 or higher, the LRS spectral extraction reference file used will have fixed coordinates specified as the extraction aperture. These coordinates are tailored to the Stage 3 Pipeline extraction (i.e. the dither-combined, resampled product), and the Level 3 products in MAST should be much improved. However, as the same extraction reference file is used for both the Stage 2 and Stage 3 iterations of ``extract_1d()``, the Level 2b extracted spectra will not be accurate. 

If you want to examine the individual nod extracted spectra, use the methods demonstrated in this notebook to create a custom extraction file tailored to the 2D spectral images you'd like to extract, and run the step standalone.



In [ ]:
import os
os.environ['CRDS_CONTEXT'] = 'jwst_1084.pmap'

In [ ]:
%matplotlib inline
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

from jwst.pipeline import Spec2Pipeline, Spec3Pipeline
from jwst import datamodels

import jwst
print('Using JWST calibration pipeline version {0}'.format(jwst.__version__))



Let's start by plotting the default Level 3 extracted output file that can be retrieved from MAST. We restrict the short-wavelength end of the x-axis to 5 micron, as our calibration is very poor below this wavelength. **This is not the spectrum we are expecting**. 

In [ ]:
l3_file = 'data/jw02072-o001_t010_miri_p750l_x1d.fits'
l3_spec = datamodels.open(l3_file)

fig, ax = plt.subplots(figsize=[12,4])
ax.plot(l3_spec.spec[0].spec_table['WAVELENGTH'], l3_spec.spec[0].spec_table['FLUX'])
ax.set_xlabel('wavelength (um)')
ax.set_ylabel('flux (Jy)')
ax.set_title('SN2021aefx - Level 3 spectrum in MAST')
ax.set_xlim(5., 14.)
fig.show()


## Spec2Pipeline

The sequence of steps for the LRS slit mode in the Spec2Pipeline is as follows:

* ``assign_wcs``: assigns the world coordinate system extension as well as the wavelength calibration
* ``background``: performs background subtraction
* ``source_type``: assigns a source type. This is inherited from APT, or if not marked in APT as unknown, it will take a pre-selected default value. For MIRI LRS the default value is ``point``. 
* ``flat_field``: divides the science data by the flat field
* ``pathloss``: corrects for wavelength-dependent flux loss due to the truncation by the slit
* ``photom``: converts DN/s pixel values to MJy/sr physical flux densities
* ``extract_1d``: performs spectral extraction. At this stage of the pipeline, the extraction is performed on each nod file separately; these are not yet the _final_ extracted products. 



### Association vs. single file input

A single ``rate`` file can be passed to this pipeline stage or any of its individual steps, and be processed successfully. However, for our default use case of a point source observation using the ALONG SLIT NOD pattern, the input to the Spec2Pipeline is an **association file**, which contains information for both files. The pipeline builds such files for all cases where multiple files need to be combined for pipeline execution. In the case of LRS slit observations, the ``background`` step performs a mutual subtraction of the nods. 

The association files used can also be downloaded from MAST. We demonstrate here how to build them from scratch using the tools provided in the pipeline package. You can read more about associations [here](https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/level2_asn_technical.html).


In [ ]:
from jwst.associations.asn_from_list import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase

In [ ]:
nod1_file = 'data/jw02072001001_06101_00001_mirimage_rate.fits'
nod2_file = 'data/jw02072001001_06101_00002_mirimage_rate.fits'

nods_list = [nod1_file, nod2_file]

In [ ]:
asn = asn_from_list(nods_list, rule=DMSLevel2bBase, meta={'program':'02072', 'target':'sn2021aefx', 'asn_pool':'None'})
print(asn)

This association now contains 2 _members_. We will provide each member exposure with its associated background, i.e. the opposite nod file, and write the association to file. 

In [ ]:
asn['products'][0]['name'] = nod1_file.split('/')[-1]
asn['products'][1]['name'] = nod2_file.split('/')[-1]
asn['products'][0]['members'].append({'expname': nod2_file, 'exptype':'background'})
asn['products'][1]['members'].append({'expname': nod1_file, 'exptype':'background'})

print(asn)

# write this out to a json file
with open('lrs_slit_test_spec2asn.json', 'w') as fp:
    fp.write(asn.dump()[1])

### Running the Spec2Pipeline


We set up and run the Spec2Pipeline, using the association file as input. 

In [ ]:
spec2 = Spec2Pipeline()
spec2.save_results = True
spec2.output_dir = 'data/'

sp2_out = spec2.run('lrs_slit_test_spec2asn.json')

By default, the Spec2Pipeline for LRS slit will return files of types: 
* ``cal.fits``: photometrically calibrated 2D spectral images, FULL array, not rectified (output from the photom step)
* ``s2d.fits``: photometrically calibrated 2D spectral images, cutout, rectified (output from the resample_spec step)
* ``x1d.fits``: 1D extracted spectra for each nod position

Let's take a look at the resampled 2d products, and at the 1D spectra that were extracted from them. **We can already see here that the automated extraction has not worked properly. The two spectra should be near-identical and they are not! Plus the fluxes are very low, when we are expecting a healthy SNR spectrum.**

In [ ]:
sp2_s2dfiles = glob('data/jw02072*s2d.fits')
#sp2_s2dfiles.reverse()
print(sp2_s2dfiles)

fig2a, ax2a = plt.subplots(ncols=2, figsize=[8,10])
for sdf, ax in zip(sp2_s2dfiles, ax2a):
    im = datamodels.open(sdf)
    ll = (sdf.split('/')[-1]).split('.')[0]
    ax.imshow(im.data, origin='lower', cmap='gist_heat', vmin=-100, vmax=100)
    ax.set_title(ll, fontsize='small')
    ax.set_xlabel('col')
    ax.set_ylabel('row')

fig2a.show()




In [ ]:
sp2_x1dfiles = glob('data/jw02072001*x1d.fits')
print(sp2_x1dfiles)
fig2b, ax2b = plt.subplots(figsize=[12,6])
for spf in sp2_x1dfiles:
    sp = datamodels.open(spf)
    print(sp.meta.ref_file.crds.context_used)
    ll = (spf.split('/')[-1]).split('.')[0]
    ax2b.plot(sp.spec[0].spec_table['WAVELENGTH'], sp.spec[0].spec_table['FLUX'], label=ll)
    
#ax2.plot(l3_spec.spec[0].spec_table['WAVELENGTH'], l3_spec.spec[0].spec_table['FLUX'], 'r-', label='default L3 spectrum')
ax2b.legend()
ax2b.set_xlabel('wavelength (um)')
ax2b.set_ylabel('flux (Jy)')
ax2b.set_xlim(5., 14.)
fig2b.show()

### Note 
The extracted products at Level 2 of the pipeline are not yet "final". They are more affected by bad pixels and detector artifacts than the final Level 3 product. They are useful for quick-look purposes or to diagnose problems, but for full science-ready spectra, you should perform the Level 3 processing. 

**The spectra we see from Level 2 are a sign that there is some coordinate issue that is preventing the automated extraction from working. In the next step we will perform the Spec3Pipeline processing and fix this issue to get a science-ready spectrum**

# Spec3Pipeline

The Level 3 pipeline combines the 2 nod images, which reduces the impact of bad pixels and provides better signal to noise; and performs the final "science-ready" extraction. Whereas the Spec2Pipeline can take both single files and associations as input, the Spec3Pipeline always requires an association as input. 

In the code below we repeat the association creation, and run the Spec3Pipeline. To ensure that the extraction aperture is placed in the correct position, we use a custom extraction reference file, ``miri_lrs_demo_extract1d.json``, that forces the pipeline to extract specific columns. You can adapt this reference file for your own science, definining custom extraction apertures and extraction limits. The file can also be used to specify:

- tapered column extraction (with linear or polynomial-shaped edges)
- background subtraction from an off-target portion of the array


and more. For full details of all the options for this reference file, see the [documentation pages](https://jwst-pipeline.readthedocs.io/en/latest/jwst/extract_1d/reference_files.html#extract1d-reference-file). 

In [ ]:
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

In [ ]:
asn3 = asn_from_list(sp2_s2dfiles, rule=DMS_Level3_Base, product_name='PID2072_Obs1_LRS_demo', meta={'program':'02072', 'target':'sn2021aefx', 'asn_pool':'None'})

with open('lrs_slit_test_spec3asn.json', 'w') as fp:
    fp.write(asn3.dump()[1])


In [ ]:
spec3 = Spec3Pipeline()
spec3.save_results = True
spec3.output_dir = 'data/'
spec3.extract_1d.override_extract1d = 'miri_lrs_demo_extract1d.json'
spec3.extract_1d.use_source_posn = False

sp3 = spec3.run('lrs_slit_test_spec3asn.json')

In [ ]:
sp3_s2dfiles = glob('data/PID2072*s2d.fits')

fig3a, ax3a = plt.subplots(figsize=[4,10])
im = datamodels.open(sp3_s2dfiles[0])
ax3a.imshow(im.data, origin='lower', cmap='gist_heat')
ax3a.set_title(sp3_s2dfiles[0].split('/')[-1])
fig3a.show()
    

In [ ]:
sp3_x1dfile = 'data/PID2072_Obs1_LRS_demo_x1d.fits'


fig3b, ax3b = plt.subplots(figsize=[12,6])

for spf in sp2_x1dfiles:
    sp = datamodels.open(spf)
    ll = (spf.split('/')[-1]).split('.')[0] + ' (Level 2b, automated)'
    ax3b.plot(sp.spec[0].spec_table['WAVELENGTH'], sp.spec[0].spec_table['FLUX'], label=ll)
    
sp3_x1d = datamodels.open(sp3_x1dfile)
ll3 = (sp3_x1dfile.split('/')[-1]).split('.')[0] + ' (Level 3, custom)'
ax3b.plot(sp3_x1d.spec[0].spec_table['WAVELENGTH'], sp3_x1d.spec[0].spec_table['FLUX'], label=ll3)

ax3b.legend()
ax3b.set_xlabel('wavelength (um)')
ax3b.set_ylabel('flux (Jy)')
ax3b.set_xlim(5., 14.)
fig3b.show()

By providing the xstart and xstop limits for the spectral extraction in the custom reference file, and setting the ``use_source_posn`` to False, we prevent the pipeline from using coordinates to place the apertue and instead force it to use our column limits. As a result we get a much cleaner spectrum. 

In the reference file update for CRDS context 1088, the ``use_source_posn`` setting is inlcluded in the extractino reference file. 

**Note**: a custom reference file can be provided for the spectral extraction in the Spec2Pipeline, exactly as we did for the Spec3Pipeline run. However this requires the ``extract_1d`` step to be run for each nod file individually, with a custom reference file adjusted to the position of the nod in the field. 

#### Please contact the [JWST Helpdesk](https://stsci.service-now.com/jwst) if you have any questions or problems!

-- END

SK, May 31st 2023

